# Feature Engineering

**Consideration for Feature Engineering:**

1. **One-Hot Encoding**: Representing categorical variables as binary features. Can help capture any non-linear relationships between the categories and the target variable.

2. **Binning or Discretization**: COnverting continuous variables into categorical variables by dividing them into bins or discrete intervals. 

3. **Feature Scaling**: Scaling numerical features to ensure that they are on a similar scale and have a similar range. *Standardization* (scaling to zero mean and unit variance) or *normalization* (scaling to a specified range, e.g., [0, 1]) can be applied to numerical features. This helps prevent features with larger values from dominating the model and ensures fair comparisons.

4. **Feature Creation**: Combine domain knowledge and insights from the EDA to identify and create potential features that may be relevant for churn prediction. 

Feature engineering is an **iterative process**, and it often requires experimentation and evaluation of different approaches. Thus, this notebook will explore 2 loops of feature engineering. Only in the second loop it will be created new features from the combination of existing ones.
**It will be evaluated the results in model performance to decide the final preprocessing code for model input.**

`Obs. Scaling will be performed to all data before model input using StandartScaler. `


## First Loop of Feature Engineering

### Actions to be performed:

**_Drop:_**
> Reason: no effect on "Exited".
> - `CustomerId`: random values of customer ID.
> - `Surname`: customer surname.


**_One-hot encoding:_**
> Reason: categorical variables
> - `Geography`: geography of the customer. 
> - `Gender`: customer gender.
> - `NumOfProducts`: number of products that a customer has purchased through the bank


**_Binning into some intervals:_**
> Reason: can help capture non-linear relationships and patterns that might not be apparent when treating the variable as continuous
> - `CreditScore`: credit score of customer.

**_No transformation for now:_**
> - `IsActiveMember`: active customers.
> - `Tenure`: years the customer is a client of the bank.
> - `HasCrCard`: denotes whether or not a customer has a credit card. 
> - `Age`: customer age. *[scaled later]*
> - `EstimatedSalary`: customer estimated salary. *[scaled later]*
> - `Balance`: balance of customer in the bank account. *[scaled later]*

In [42]:
import pandas as pd
import numpy as np

In [49]:
# Load data
df = pd.read_csv('../data/01_raw/abandono_clientes.csv')

# Drop irrelevant variables
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Binning 
#-- CreditScore column
bins = [350, 500, 650, 800, 850]  # Define the bin edges
labels = ['Low', 'Medium', 'High', 'Very High']  # Define the bin labels

df['CreditScoreBins'] = pd.cut(df['CreditScore'], bins=bins, labels=labels)

# One hot encoding for categorical variables
df = pd.get_dummies(df, columns =['Geography', 'Gender', 'CreditScoreBins', 'NumOfProducts'], drop_first = True)

# Save to csv
df.to_csv('../data/04_feature/features_first_loop.csv', index=False)

df.head()


,CreditScore,Age,Tenure,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male,CreditScoreBins_Medium,CreditScoreBins_High,CreditScoreBins_Very High,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
0,619,42,2,0.00,1,1,101348.88,1,False,False,False,True,False,False,False,False,False
1,608,41,1,83807.86,0,1,112542.58,0,False,True,False,True,False,False,False,False,False
2,502,42,8,159660.80,1,0,113931.57,1,False,False,False,True,False,False,False,True,False
3,699,39,1,0.00,0,0,93826.63,0,False,False,False,False,True,False,True,False,False
4,850,43,2,125510.82,1,1,79084.10,0,False,True,False,False,False,True,False,False,False


In [45]:
df.describe()

,Tenure,Balance,HasCrCard,IsActiveMember,Exited
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,5.012800,76485.889288,0.70550,0.515100,0.203700
std,2.892174,62397.405202,0.45584,0.499797,0.402769
min,0.000000,0.000000,0.00000,0.000000,0.000000
25%,3.000000,0.000000,0.00000,0.000000,0.000000
50%,5.000000,97198.540000,1.00000,1.000000,0.000000
75%,7.000000,127644.240000,1.00000,1.000000,0.000000
max,10.000000,250898.090000,1.00000,1.000000,1.000000


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Tenure                     10000 non-null  int64  
 1   Balance                    10000 non-null  float64
 2   HasCrCard                  10000 non-null  int64  
 3   IsActiveMember             10000 non-null  int64  
 4   Exited                     10000 non-null  int64  
 5   Geography_Germany          10000 non-null  bool   
 6   Geography_Spain            10000 non-null  bool   
 7   Gender_Male                10000 non-null  bool   
 8   CreditScoreBins_Medium     10000 non-null  bool   
 9   CreditScoreBins_High       10000 non-null  bool   
 10  CreditScoreBins_Very High  10000 non-null  bool   
 11  NumOfProducts_2            10000 non-null  bool   
 12  NumOfProducts_3            10000 non-null  bool   
 13  NumOfProducts_4            10000 non-null  bool

## Second Loop:

From EDA we were able to extranct some insights to formulate new variables:

- "Balance" varies significantly by "Geography". From EDA avg for Germany is around 130k, while for Spain and France is around 60k. Lets create a variable "Balance_GreographyMean_Ratio".
- Estimated Salary and Credit Score could be related
- Age and Tenure can have a relationship.


**_Features to be created:_**
> - *EstimatedSalary_CreditScore_Ratio*
> - *CreditScore_EstimatedSalary_Ratio*
> - *Balance_EstimatedSalary_Ratio*
> - *Balance_IsZero*
> - *Balance_GeographyMean_Ratio*
> - *Tenure_Age_Ratio*
> - *NumOfProducts_Tenure_Ratio*

In [54]:
# Load data
df = pd.read_csv('../data/01_raw/abandono_clientes.csv')

# When the division is 0, resulting variable will be zero:
# EstimatedSalary / CreditScore
df['EstimatedSalary_CreditScore_Ratio'] = np.where(df['CreditScore'] == 0, 0, df['EstimatedSalary'] / df['CreditScore'])

# CreditScore / EstimatedSalary
df['CreditScore_EstimatedSalary_Ratio'] = np.where(df['EstimatedSalary'] == 0, 0, df['CreditScore'] / df['EstimatedSalary'])

# Balance / EstimatedSalary
df['Balance_EstimatedSalary_Ratio'] = np.where(df['EstimatedSalary'] == 0, 0, df['Balance'] / df['EstimatedSalary'])

# Balance is zero
df['Balance_IsZero'] = df['Balance'] == 0

# Balance / Geography Mean Balance
mean_balance_germany = 120000 # from EDA
mean_balance_spain_france = 60000 

df['Balance_GeographyMean_Ratio'] = df['Balance'] / np.where(df['Geography'] == 'Germany', mean_balance_germany, mean_balance_spain_france)

# Tenure / Age
df['Tenure_Age_Ratio'] = np.where(df['Age'] == 0, 0, df['Tenure'] / df['Age'])

# NumOf / Age
df['NumOfProducts_Tenure_Ratio'] = np.where(df['Tenure'] == 0, 0, df['NumOfProducts'] / df['Tenure'])


#########
# From the first feature engineering loop:
#########

# Drop irrelevant variables
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Binning 
#-- CreditScore column
bins = [350, 500, 650, 800, 850]  # Define the bin edges
labels = ['Low', 'Medium', 'High', 'Very High']  # Define the bin labels

df['CreditScoreBins'] = pd.cut(df['CreditScore'], bins=bins, labels=labels)

# One hot encoding for categorical variables
df = pd.get_dummies(df, columns =['Geography', 'Gender', 'CreditScoreBins', 'NumOfProducts'], drop_first = True)
#########

# Save to csv
df.to_csv('../data/04_feature/features_second_loop.csv', index=False)

df.head()


,CreditScore,Age,Tenure,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,EstimatedSalary_CreditScore_Ratio,CreditScore_EstimatedSalary_Ratio,...,NumOfProducts_Tenure_Ratio,Geography_Germany,Geography_Spain,Gender_Male,CreditScoreBins_Medium,CreditScoreBins_High,CreditScoreBins_Very High,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
0,619,42,2,0.00,1,1,101348.88,1,163.730016,0.006108,...,0.500,False,False,False,True,False,False,False,False,False
1,608,41,1,83807.86,0,1,112542.58,0,185.102928,0.005402,...,1.000,False,True,False,True,False,False,False,False,False
2,502,42,8,159660.80,1,0,113931.57,1,226.955319,0.004406,...,0.375,False,False,False,True,False,False,False,True,False
3,699,39,1,0.00,0,0,93826.63,0,134.229800,0.007450,...,2.000,False,False,False,False,True,False,True,False,False
4,850,43,2,125510.82,1,1,79084.10,0,93.040118,0.010748,...,0.500,False,True,False,False,False,True,False,False,False


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   CreditScore                        10000 non-null  int64  
 1   Age                                10000 non-null  int64  
 2   Tenure                             10000 non-null  int64  
 3   Balance                            10000 non-null  float64
 4   HasCrCard                          10000 non-null  int64  
 5   IsActiveMember                     10000 non-null  int64  
 6   EstimatedSalary                    10000 non-null  float64
 7   Exited                             10000 non-null  int64  
 8   EstimatedSalary_CreditScore_Ratio  10000 non-null  float64
 9   CreditScore_EstimatedSalary_Ratio  10000 non-null  float64
 10  Balance_EstimatedSalary_Ratio      10000 non-null  float64
 11  Balance_IsZero                     10000 non-null  bool

_Author: Laiza Milena Scheid Parizotto_

_Date: June 19th, 2023_